In [ ]:
import sys
sys.path.append(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data_science_code')

from data_processor import DataProcessor


In [ ]:
data_processor = DataProcessor()
train_df = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\train.csv')
test_df = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\test.csv')
gender_submission = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\gender_submission.csv')

In [ ]:
train_data, test_data = data_processor.train_test_split(train_df, test_size=0.2, random_state=42)
